<a href="https://colab.research.google.com/github/sophia-zhang-qwq/animal-pose-est/blob/main/sleap_environment_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1></h1>
<h2>SLEAP TRAINING AND INFERENCE GUIDE</h2>
</center>
<center>Delia Curran</center>


<center>April 2, 2025 </center>

# SLEAP Requirements

---
## SLEAP Software Requirements:
*   All training videos must have the same aspect ratio
  *   *see: [https://github.com/talmolab/sleap/issues/2004](https://github.com/talmolab/sleap/issues/2004)*
*   Input scaling must be set to 1
  *   *see: [https://github.com/talmolab/sleap/discussions/925](https://github.com/talmolab/sleap/discussions/925)*
*   Python version must be <=3.10
  *   *see: [https://github.com/talmolab/sleap/issues/2100](https://github.com/talmolab/sleap/issues/2100)*
  *   *guide:[https://gist.github.com/kargaranamir/e0b7910fed0a31](https://gist.github.com/kargaranamir/e0b7910fed0a3189563d9254c7a2c439)*
*   Must use CUDA 11.2 and cuDNN 8.1
  *   *see: [https://github.com/DeepLabCut/DeepLabCut/issues/2465](https://github.com/DeepLabCut/DeepLabCut/issues/2465)*
---

---
## Ongoing Issues:
*   Issues with inference ([https://github.com/talmolab/sleap/issues/628](https://github.com/talmolab/sleap/issues/628))
---

# Configure Training Environment

In [3]:
# override and install python 3.10
! wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
! chmod +x mini.sh
! bash ./mini.sh -b -f -p /usr/local
! conda install -q -y jupyter
! conda install -q -y google-colab -c conda-forge
! python -m ipykernel install --name "py310" --user

--2025-04-29 18:08:42--  https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72402405 (69M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  69.05M   146MB/s    in 0.5s    

2025-04-29 18:08:42 (146 MB/s) - ‘mini.sh’ saved [72402405/72402405]

PREFIX=/usr/local
Unpacking payload ...
                                                                            
Installing base environment...





Preparing transaction: - \ | / - \ done
Executing transaction: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected b

# Install SLEAP & Dependencies

In [4]:
# confirm using python 3.10
! python3 --version

Python 3.10.16


In [5]:
# install sleap and dependencies
! pip install sleap[pypi]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.1/904.1 kB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 101.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# install additional sleap dependencies and load GPU support through CUDA
! pip install matplotlib-inline
! pip install ipython
! apt-get install cuda-11-8
! apt-get install -y libcudnn8=8.6.0.163-1+cuda11.8
! pip install numpy==1.23

! export PATH=/usr/local/cuda-11.8/bin:$PATH
! export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH
! export CUDNN_INCLUDE_DIR=/usr/local/cuda/include
! export CUDNN_LIB_DIR=/usr/local/cuda/lib64

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cpp-12 cuda-cccl-11-8 cuda-command-line-tools-11-8 cuda-compiler-11-8
  cuda-cudart-11-8 cuda-cudart-dev-11-8 cuda-cuobjdump-11-8 cuda-cupti-11-8
  cuda-cupti-dev-11-8 cuda-cuxxfilt-11-8 cuda-demo-suite-11-8
  cuda-documentation-11-8 cuda-driver-dev-11-8 cuda-drivers cuda-drivers-570
  cuda-gdb-11-8 cuda-libraries-11-8 cuda-libraries-dev-11-8 cuda-memcheck-11-8
  cuda-nsight-11-8 cuda-nsight-compute-11-8 cuda-nsight-systems-11-8
  cuda-nvcc-11-8 cuda-nvdisasm-11-8 cuda-nvml-dev-11-8 cuda-nvprof-11-8
  cuda-nvprune-11-8 cuda-nvrtc-11-8 cuda-nvrtc-dev-11-8 cuda-nvtx-11-8
  cuda-nvvp-11-8 cuda-profiler-api-11-8 cuda-runtime-11-8 cuda-sanitizer-11-8
  cuda-toolkit-11-8 cuda-toolkit-11-8-config-common
  cuda-toolkit-11-config-common cuda-tools-11-8 cuda-visual-tools-11-8
  dctrl-tools default-jre default-jre-headless dkms fakeroot fonts-deja

In [ ]:
# restart runtime so GPU changes take effect
import os
os.kill(os.getpid(), 9)

# Run SLEAP Training

In [1]:
# load sleap training files from google drive
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/sleap/mice_hc")
#! unzip model.slp.training_job.zip
! ls
print("Successfully loaded sleap training files")

Mounted at /content/drive/
colab.predicted_suggestions.slp  mice_hc.topdown.predictions.slp
inference_times_hist.png	 mice_hc.train.pkg.slp
inference_times_plot.png	 mini.sh
mice_hc.demo.mp4		 models
mice_hc.demo.predictions.slp	 video_output.mp4
Successfully loaded sleap training files


In [2]:
# keep the sleap program running in background
import threading
import time
def keep_colab_alive():
    while True:
        print("Keeping Colab Session Active...")
        time.sleep(600)
threading.Thread(target=keep_colab_alive, daemon=True).start()

Keeping Colab Session Active...


In [4]:
# run top-down sleap training using loaded training job
#! sleap-train centroid.json trained_model.pkg.slp
#! sleap-train centered_instance.json trained_model.pkg.slp

#FileNotFoundError: Could not find training profile: centroid.json
! sleap-train baseline.centroid.json mice_hc.train.pkg.slp --run_name "mice_hc.single.centroid"

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:sleap.nn.training:Versions:
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:sleap.nn.training:Training labels file: mice_hc.train.pkg.slp
INFO:sleap.nn.training:Training profile: /usr/local/lib/python3.10/site-packages/sleap/training_profiles/baseline.centroid.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline.centroid.json",
    "labels_path": "mice_hc.train.pkg.slp",
    "video_paths": [
        ""
    ],
    "val_labels": null,
    "test_labels": null,
    "base_checkpoint": null,
    "tensorboard": false,
    "save_viz": false,
    "keep_viz": false,
    "zmq": false,
    "publish_port": 9001,
    "controller_port": 9000,
    "run_name": "mice_hc.single.centroid",
    "prefix": "",
    "suffix": "",
    "cpu": false,
    "first_gpu": false,
    "last_gpu": false,
    "gpu": "auto"
}
INFO:sleap.nn.

In [3]:
# run top-down sleap training using loaded training job
#! sleap-train centroid.json trained_model.pkg.slp
#! sleap-train centered_instance.json trained_model.pkg.slp

#FileNotFoundError: Could not find training profile: centered_instance.json
! sleap-train baseline_large_rf.single.json mice_hc.train.pkg.slp --run_name "mice_hc.single.centered_instance"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:matplotlib.font_manager:generated new fontManager
INFO:sleap.nn.training:Versions:
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:sleap.nn.training:Training labels file: mice_hc.train.pkg.slp
INFO:sleap.nn.training:Training profile: /usr/local/lib/python3.10/site-packages/sleap/training_profiles/baseline_large_rf.single.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline_large_rf.single.json",
    "labels_path": "mice_hc.train.pkg.slp",
    "video_paths": [
        ""
    ],
    "val_labels": null,
    "test_labels": null,
    "base_checkpoint": null,
    "tensorboard": false,
    "save_viz": false,
    "keep_viz": false,
    "zmq": false,
    "publish_port": 9001,
    "controller_port": 9000,
    "run_name": "mice_hc.single.centered_instance",
    "prefix": "",
    "suffix": "",
    "cpu": false,
   

In [4]:
# run multi-class top-down-id sleap training using loaded training job (instance tracking)
# no need to re-train this
#! sleap-train centroid.json trained_model.pkg.slp
#! sleap-train multi_class_topdown.json trained_model.pkg.slp
#baseline_large_rf
! sleap-train baseline_large_rf.topdown.json mice_hc.train.pkg.slp --run_name "mice_hc.multi_class.topdown"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:sleap.nn.training:Versions:
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:sleap.nn.training:Training labels file: mice_hc.train.pkg.slp
INFO:sleap.nn.training:Training profile: /usr/local/lib/python3.10/site-packages/sleap/training_profiles/baseline_large_rf.topdown.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline_large_rf.topdown.json",
    "labels_path": "mice_hc.train.pkg.slp",
    "video_paths": [
        ""
    ],
    "val_labels": null,
    "test_labels": null,
    "base_checkpoint": null,
    "tensorboard": false,
    "save_viz": false,
    "keep_viz": false,
    "zmq": false,
    "publish_port": 9001,
    "controller_port": 9000,
    "run_name": "mice_hc.multi_class.topdown",
    "prefix": "",
    "suffix": "",
    "cpu": false,
    "first_gpu": false,
    "last_gpu": false,
    "gpu": "au

# Run SLEAP Inference

In [18]:
# load sleap inference files from google drive
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/sleap/mice_hc/models"
!ls "/content/drive/My Drive/sleap/mice_hc"
print("Successfully loaded sleap inference files")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mice_hc.bottomup		  mice_hc.single.centered_instance_1
mice_hc.centroid		  mice_hc.single.centroid
mice_hc.multi_class.topdown	  mice_hc.topdown
mice_hc.single.centered_instance
colab.predicted_suggestions.slp  mice_hc.track.slp
inference_times_hist.png	 mice_hc.train.pkg.slp
inference_times_plot.png	 mini.sh
mice_hc.demo.mp4		 models
mice_hc.demo.predictions.slp	 video_output.mp4
mice_hc.topdown.predictions.slp
Successfully loaded sleap inference files


In [21]:
!ls
!apt-get install tree
!tree /content/drive/MyDrive/sleap/mice_hc

colab.predicted_suggestions.slp  mice_hc.track.slp
inference_times_hist.png	 mice_hc.train.pkg.slp
inference_times_plot.png	 mini.sh
mice_hc.demo.mp4		 models
mice_hc.demo.predictions.slp	 video_output.mp4
mice_hc.topdown.predictions.slp
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
/content/drive/MyDrive/sleap/mice_hc
├── colab.predicted_suggestions.slp
├── inference_times_hist.png
├── inference_times_plot.png
├── mice_hc.demo.mp4
├── mice_hc.demo.predictions.slp
├── mice_hc.topdown.predictions.slp
├── mice_hc.track.slp
├── mice_hc.train.pkg.slp
├── mini.sh
├── models
│   ├── mice_hc.bottomup
│   │   ├── best_model.h5
│   │   ├── initial_config.json
│   │   ├── labels_gt.train.slp
│   │   ├── labels_gt.val.slp
│   │   ├── labels_pr.train.slp
│   │   ├── labels_pr.val.slp
│   │   ├── metrics.train.npz
│   │   ├── metrics.val.np

In [59]:
!python -c "import sleap; curr = sleap.load_file('mice_hc.train.pkg.slp'); print(curr); sugg=curr.suggestions; print(f'Number of suggested frames: {len(sugg)}') "

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
/usr/local/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
Labels(labeled_frames=1178, videos=40, skeletons=1, tracks=98)
Number of suggested frames: 0


In [56]:
# sleap-track := apply trained SLEAP models to predict instances in video or slp files

# predicting instances in suggested frames from exported .slp file
# run the SLEAP tracking tool, using two models (centroid and instance),
# only on the suggested frames of a given .slp file,
# with a detection threshold of 0.05,
# saves the resulting predictions to file name called "001.predictions.slp."

# arg:
#   --threshold 0.05 := sets the detection threshold for centroid detection to 0.05
# low val := more sensitive, detect low-confidence anmials, high val := more strict
# 0.05 is fairly low
#   -o 001.predictions.slp := output name is "001.predictions.slp"
#   /content/drive/My\ Drive/sleap/trained_model.pkg.slp := input .slp file
# sleap looks at the suggested fames in this file and apply the models to predict instances on those frames
"""
!sleap-track \
    -m /content/drive/My\ Drive/sleap/models/top-down250401_145841.centroid \
    -m /content/drive/My\ Drive/sleap/models/top-down250401_145841.centered_instance \
    --only-suggested-frames \
    --threshold 0.05 \
    -o 001.predictions.slp \
    /content/drive/My\ Drive/sleap/trained_model.pkg.slp
"""

!sleap-track \
    -m models/mice_hc.single.centroid \
    -m models/mice_hc.single.centered_instance \
    -o mice_hc_single_predictions.slp \
    mice_hc.train.pkg.slp

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Started inference at: 2025-04-29 19:26:06.176399
Args:
{
│   'data_path': 'mice_hc.train.pkg.slp',
│   'models': [
│   │   'models/mice_hc.single.centroid',
│   │   'models/mice_hc.single.centered_instance'
│   ],
│   'frames': '',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': 'mice_hc_single_predictions.slp',
│   'no_empty_frames': False,
│   'verbosity': 'rich',
│   'video.dataset': None,
│   'video.input_format': 'channels_last',
│   'video.index': '',
│   'cpu': False,
│   'first_gpu': False,
│   'last_gpu': False,
│   'gpu': 'auto',
│   'max_edge_length_ratio': 0.25,
│   'dist_penalty_weight': 1.0,
│   'batch_size': 4,
│   'open_in_gui': False,
│   'peak_threshold': 0.2,
│   'max_instances': None,
│   'tracking.tracker': None,
│   'tracking.max_tracking': None,
│   'tracking.max_tracks': None,
│   'tracking.target_instance_count': None,
│   'tracking.pre_cull_to_target': None,
│   'tracking.p

In [62]:
# if running top-down-id
# -m /content/drive/My\ Drive/Academic/sleap/models/top-down250401_145841.multi_class_topdown \

!sleap-track \
    -m models/mice_hc.multi_class.topdown \
    -o mice_hc_multi_class_predictions.slp \
    mice_hc.train.pkg.slp

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Started inference at: 2025-04-29 19:33:00.027232
Args:
{
│   'data_path': 'mice_hc.train.pkg.slp',
│   'models': ['models/mice_hc.multi_class.topdown'],
│   'frames': '',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': 'mice_hc_multi_class_predictions.slp',
│   'no_empty_frames': False,
│   'verbosity': 'rich',
│   'video.dataset': None,
│   'video.input_format': 'channels_last',
│   'video.index': '',
│   'cpu': False,
│   'first_gpu': False,
│   'last_gpu': False,
│   'gpu': 'auto',
│   'max_edge_length_ratio': 0.25,
│   'dist_penalty_weight': 1.0,
│   'batch_size': 4,
│   'open_in_gui': False,
│   'peak_threshold': 0.2,
│   'max_instances': None,
│   'tracking.tracker': None,
│   'tracking.max_tracking': None,
│   'tracking.max_tracks': None,
│   'tracking.target_instance_count': None,
│   'tracking.pre_cull_to_target': None,
│   'tracking.pre_cull_iou_threshold': None,
│   'tracking.post_connect

In [64]:
# predicting and tracking instances in uploaded video (example.mp4)
# ffmpeg := perform a set of video preprocessing steps,
# -i input file, -vf video filter and is used to apply transformations to the video
# rescake the video to a resolution of 1280x720 pixels with some paddings
# output the name as "crop_example.mp4"

# then run inference(sleap-track) on the crop_example.mp4 video
# Saved output: crop_mice_hc.demo.predictions.slp
! ffmpeg -i 'mice_hc.demo.mp4' -vf "scale=1280:720:force_original_aspect_ratio=decrease,pad=1280:720:(ow-iw)/2:(oh-ih)/2:black" 'crop_mice_hc.demo.mp4'
!sleap-track  crop_mice_hc.demo.mp4\
    --frames 0-100 \
    --tracking.tracker simple \
    -m models/mice_hc.single.centroid \
    -m models/mice_hc.single.centered_instance

ffmpeg: /usr/local/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /usr/local/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --

# Testing

In [6]:
!python -c "import sleap; sleap.disable_preallocation(); print(sleap.versions()); sleap.system_summary()"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:matplotlib.font_manager:generated new fontManager
2025-04-29 18:30:53.798408: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-29 18:30:55.126449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-29 18:30:55.126815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
None
GPUs: 1/1 available
  Device: /physical_device:GPU:0
         Available: True
       Initialized: False
     Memory growth: Tr